In [1]:
# from platform import python_version
# print(python_version())

In [2]:
# !pip install tensorflow==1.13.1
# !pip install Sequential
# !pip install keras

In [3]:
# Import Required Libraries
import tensorflow as tf
from tensorflow import data
from tensorflow.data import Dataset

from tensorflow import keras 
from keras.layers import Dense, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

import os
import json
import math
import numpy as np

import sentiment_dataset as sentiment_dataset
import sentiment_model_cnn as sentiment_model_cnn
import config_holder as config_holder

Using TensorFlow backend.


In [4]:
# Specify Parameters
num_epoch= 10
config_file = json.load(open('training_config.json', "r"))
bucket_location = 'ieor4577-hw6'
key_train = "train"
key_dev = "dev"
key_eval = "eval"

In [ ]:
# Train CNN Model
print("Preparing for training...")

training_config = config_file

training_config["num_epoch"] = num_epoch

train_dataset = sentiment_dataset.train_input_fn(bucket_location, key_train, training_config)
validation_dataset = sentiment_dataset.validation_input_fn(bucket_location, key_dev, training_config)
eval_dataset = sentiment_dataset.eval_input_fn(bucket_location, key_eval, training_config)

model = sentiment_model_cnn.keras_model_fn(None, training_config)

print("Starting training...")
validation_dataset[1]
model.fit(
    x=train_dataset[0], y=train_dataset[1], steps_per_epoch=train_dataset[2]["num_batches"],
    epochs=training_config["num_epoch"],
    validation_data=(validation_dataset[0], validation_dataset[1]),
    validation_steps=int(validation_dataset[2]["num_batches"]))

score = model.evaluate(eval_dataset[0], eval_dataset[1], steps=eval_dataset[2]["num_batches"], verbose=0)

print("Test loss:{}".format(score[0]))
print("Test accuracy:{}".format(score[1]))

Preparing for training...
Fetching train data...
Fetching validation data...
Fetching eval data...
Instructions for updating:
Colocations handled automatically by placer.
Starting training...
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
13600/13600 [==============================] - 362s 27ms/step - loss: 0.4373 - acc: 0.7944 - val_loss: 0.4124 - val_acc: 0.8091
Epoch 2/10
13600/13600 [==============================] - 360s 26ms/step - loss: 0.4012 - acc: 0.8159 - val_loss: 0.4084 - val_acc: 0.8114
Epoch 3/10
13600/13600 [==============================] - 356s 26ms/step - loss: 0.3856 - acc: 0.8248 - val_loss: 0.4042 - val_acc: 0.8139
Epoch 4/10
13600/13600 [==============================] - 359s 26ms/step - loss: 0.3729 - acc: 0.8319 - val_loss: 0.4039 - val_acc: 0.8154
Epoch 5/10
13600/13600 [==============================] - 358s 26ms/step - loss: 0.3619 - acc: 0.8385 - val_loss: 0.4046 - val

In [ ]:
# Save Trained Model
tf.contrib.saved_model.save_keras_model(model, "sentiment_model.h5")


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.


Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensor

b'sentiment_model.h5/1583604702'

In [8]:
# Save Results to S3
import boto3

s3 = boto3.client('s3')
s3.put_object(Bucket="ieor4577-hw6", Key=("sentiment_model.h5/1/assets"+'/'))
s3.put_object(Bucket="ieor4577-hw6", Key=("sentiment_model.h5/1/variables"+'/'))

s3 = boto3.resource('s3')
s3.meta.client.upload_file('sentiment_model.h5/1/saved_model.pb', 
                           'ieor4577-hw6', 
                           'sentiment_model.h5/1/saved_model.pb')
s3.meta.client.upload_file('sentiment_model.h5/1/assets/saved_model.json', 
                           'ieor4577-hw6', 
                           'sentiment_model.h5/1/assets/saved_model.json')
s3.meta.client.upload_file('sentiment_model.h5/1/variables/checkpoint', 
                           'ieor4577-hw6', 
                           'sentiment_model.h5/1/variables/checkpoint')
s3.meta.client.upload_file('sentiment_model.h5/1/variables/variables.data-00000-of-00001', 
                           'ieor4577-hw6', 
                           'sentiment_model.h5/1/variables/variables.data-00000-of-00001')
s3.meta.client.upload_file('sentiment_model.h5/1/variables/variables.index', 
                           'ieor4577-hw6', 
                           'sentiment_model.h5/1/variables/variables.index')

print("Model successfully saved")

Model successfully saved
